In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import EfficientNetB0
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

# Load the data from the Excel file
data = pd.read_excel('C:/Users/DELL/Documents/IAS internship docs/IAS project/Hypertensive retinopathy/Dataset/2-Hypertensive Retinopathy Classification/2-Groundtruths/HRDC Hypertensive Retinopathy Classification Training Labels.xlsx', engine='openpyxl')

# Convert the column values to strings
data['Hypertensive Retinopathy'] = data['Hypertensive Retinopathy'].astype(str)

# Split the data into training and validation sets
train_data, valid_data = train_test_split(data, test_size=0.2, random_state=42)

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='C:/Users/DELL/Documents/IAS internship docs/IAS project/Hypertensive retinopathy/Dataset/2-Hypertensive Retinopathy Classification/1-Images/1-Training Set',
    x_col='Image',
    y_col='Hypertensive Retinopathy',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_data,
    directory='C:/Users/DELL/Documents/IAS internship docs/IAS project/Hypertensive retinopathy/Dataset/2-Hypertensive Retinopathy Classification/1-Images/1-Training Set',
    x_col='Image',
    y_col='Hypertensive Retinopathy',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Load the EfficientNetB0 model without the top (fully connected) layers
efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers so they are not trained during the fine-tuning process
for layer in efficientnet_model.layers:
    layer.trainable = False

# Add custom fully connected layers on top of the EfficientNet base model
model = Sequential()
model.add(efficientnet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, validation_data=valid_generator, epochs=100)

# Evaluate the model on the test set
test_generator = valid_datagen.flow_from_dataframe(
    dataframe=data,
    directory='C:/Users/DELL/Documents/IAS internship docs/IAS project/Hypertensive retinopathy/Dataset/2-Hypertensive Retinopathy Classification/1-Images/1-Training Set',
    x_col='Image',
    y_col='Hypertensive Retinopathy',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Found 569 validated image filenames belonging to 2 classes.
Found 143 validated image filenames belonging to 2 classes.
16705208/16705208 [==============================] - 23s 1us/step


c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/10
18/18 [==============================] - 68s 3s/step - loss: 0.7045 - accuracy: 0.5764 - val_loss: 0.7134 - val_accuracy: 0.4266
Epoch 2/10
18/18 [==============================] - 54s 3s/step - loss: 0.6851 - accuracy: 0.5729 - val_loss: 0.6851 - val_accuracy: 0.5734
Epoch 3/10
18/18 [==============================] - 55s 3s/step - loss: 0.7131 - accuracy: 0.5782 - val_loss: 0.7814 - val_accuracy: 0.4266
Epoch 4/10
18/18 [==============================] - 56s 3s/step - loss: 0.7265 - accuracy: 0.5272 - val_loss: 0.7144 - val_accuracy: 0.4266
Epoch 5/10
18/18 [==============================] - 53s 3s/step - loss: 0.7035 - accuracy: 0.5466 - val_loss: 0.6825 - val_accuracy: 0.5734
Epoch 6/10
18/18 [==============================] - 55s 3s/step - loss: 0.6790 - accuracy: 0.5940 - val_loss: 0.7210 - val_accuracy: 0.5734
Epoch 7/10
18/18 [==============================] - 55s 3s/step - loss: 0.6835 - accuracy: 0.5571 - val_loss: 0.6868 - val_accuracy: 0.5734
Epoch 8/10
18/18 [==